In [45]:
import pretty_midi
import glob
import pickle

from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os
import matplotlib.cm as cm

In [63]:
def get_filename(filepath):
    base = os.path.basename(filepath)
    return os.path.splitext(base)[0]

In [67]:
NPITCH = 128
# output_t_size = 500 = 5 * 100 bc. 100 is default freq pretty_midi
def export_piano_rolls(roll, filename, output_t_size=500):
    print("exporting ", filename)
    
    pitch, ntime = piano_rolls[0].shape
    if pitch != NPITCH:
        print("Mismatch number of pitches")

    for st in range(0, ntime, output_t_size):
        roll_img = roll[:,st:st+output_t_size]
        if st + output_t_size > ntime:
            break
        plt.imsave('rolls_imgs/{}-st-{}.png'.format(filename, st), roll_img, cmap=cm.gray)
    print("Exported the song {}".format(filename))

In [68]:
def get_piano_rolls():
    pr = []
    done = False
    for file in glob.glob("../type0/*.midi"):
        pm = pretty_midi.PrettyMIDI(file) #takes a midi file and converts to pretty_midi
        instr = pm.instruments #splits into list of instruments
        for instrument in instr:
            name = pretty_midi.program_to_instrument_name(instrument.program)
            if name == "Acoustic Grand Piano": #only take the piano track
                piano_roll = instrument.get_piano_roll() #get the piano roll, which is a np.ndarray
                pr.append(piano_roll)  
                roll = (piano_roll[:,:]>0).astype(int)
                filename = get_filename(file)
                export_piano_rolls(roll, filename)
                if len(pr) == 1:
                    done = True
                    break
        if done:
            break
    return pr #list of piano rolls (np.ndarray matrices)

In [69]:
get_piano_rolls()

exporting  beethoven_opus22_2_format0
Exported the song beethoven_opus22_2_format0


[array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])]